# 샘플 01: OpenAI SDK를 통한 빠른 채팅

이 노트북은 Microsoft Foundry Local과 함께 OpenAI SDK를 사용하여 빠른 채팅 상호작용을 구현하는 방법을 보여줍니다.

## 개요

이 예제에서는 다음을 보여줍니다:
- Foundry Local과 함께 OpenAI Python SDK 사용
- Azure OpenAI와 로컬 Foundry 설정 모두 처리
- 적절한 오류 처리 및 대체 전략 구현
- 서비스 관리를 위한 FoundryLocalManager 사용


## 사전 준비 사항

다음 사항을 확인하세요:
1. Foundry Local이 설치되어 실행 중인지
2. 모델이 로드되어 있는지 (예: `foundry model run phi-4-mini`)
3. 필요한 Python 패키지가 설치되어 있는지

### 종속성 설치


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## 필수 라이브러리 가져오기


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## 구성

구성을 설정하세요. 다음 중 하나를 사용할 수 있습니다:
1. **Azure OpenAI** (클라우드 기반)
2. **Foundry Local** (로컬 추론)

### 옵션 1: Azure OpenAI 구성


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### 옵션 2: 파운드리 로컬 구성


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## 클라이언트 초기화

구성에 따라 OpenAI 클라이언트를 생성합니다:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## 간단한 채팅 예제

간단한 채팅 상호작용을 테스트해봅시다:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## 대화형 채팅 세션

다양한 유형의 프롬프트를 시도하여 모델이 어떻게 반응하는지 확인해 보세요:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## 고급 사용법: 스트리밍 응답

응답이 길어질 경우, 스트리밍은 더 나은 사용자 경험을 제공할 수 있습니다:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## 오류 처리 및 진단

서비스 상태와 사용 가능한 모델을 확인해봅시다:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## 사용자 지정 프롬프트 테스트

아래 셀을 사용하여 자신만의 프롬프트를 테스트하세요:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## 요약

이 노트북에서는 다음을 시연했습니다:

1. **✅ 클라이언트 설정**: Foundry Local을 사용하여 OpenAI SDK를 구성하는 방법
2. **✅ 기본 채팅**: 간단한 요청-응답 상호작용
3. **✅ 스트리밍**: 실시간 응답 생성
4. **✅ 오류 처리**: 견고한 오류 처리 및 진단
5. **✅ 서비스 상태**: 모델 가용성과 서비스 상태 확인

### 다음 단계

- **샘플 02**: Azure OpenAI 지원을 포함한 고급 SDK 통합
- **샘플 04**: Chainlit 채팅 애플리케이션 구축
- **샘플 05**: 다중 에이전트 오케스트레이션 시스템
- **샘플 06**: 지능형 모델 라우팅

### Foundry Local의 주요 장점

- 🔒 **프라이버시**: 데이터가 절대 기기를 떠나지 않음
- ⚡ **속도**: 낮은 지연 시간의 로컬 추론
- 💰 **비용**: API 사용 비용 없음
- 🔌 **오프라인**: 인터넷 연결 없이 작동
- 🛠️ **호환성**: OpenAI 호환 API
